In [15]:
%matplotlib inline

In [16]:
%run ./setup_notebook.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
# notebook specific imports
import pickle
import gsheets_importer
import math
import copy
from scipy import stats
import time
import random

In [18]:
packerstation_path = '/home/jamesrowland/Documents/packerstation/jrowland/Data'
pkl_path = '/home/jamesrowland/Documents/Code/Vape/run_pkls/RL048/run28.pkl'

with open(pkl_path, 'rb') as f:
    run = pickle.load(f)

In [6]:
figure_path = '/home/jamesrowland/Documents/Code/Vape/figures/s1s2_imaging_subsets_prototype'

In [111]:
num_planes = run.num_planes
utils.correct_s2p_combined(run.s2p_path, num_planes)
combined_path = os.path.join(run.s2p_path, 'combined')

flu, spks, stat = utils.s2p_loader(combined_path)
t1 = time.time()
flu = utils.dfof2(flu)
t2 = time.time()

n_cells = flu.shape[0]
fs = 5 # imaging samping rate (read this from suite2p in the future)

flu.shape

Loading 2727 traces labelled as cells
Subtracting neuropil with a coefficient of 0.7


(2727, 23936)

In [43]:
# the lengths of the actual tseries
tseries_lens = run.num_frames
# get the frame ttls recorded in paqio that actually correspond to analysed tseries
paqio_frames = utils.tseries_finder(tseries_lens, run.frame_clock)

if len(paqio_frames) == sum(tseries_lens):
    print('All tseries chunks found in frame clock')
else:
    print('WARNING: Could not find all tseries chunks in frame clock, check this')

if flu.shape[1] * num_planes >= len(paqio_frames) and \
   flu.shape[1] * num_planes <= len(paqio_frames) + num_planes:
    print('Fluorescence matrix shape matches recorded frame clock')
else:
    print('WARNING: Fluoresence matrix does not match frame clock shape, check this')

All tseries chunks found in frame clock
Fluorescence matrix shape matches recorded frame clock


In [124]:
# convert paqio recorded frames to pycontrol ms
ms_vector = run.both_aligner.B_to_A(paqio_frames) # flat list of plane times

# which plane is each cell in
cell_plane = np.array([s['iplane'] for s in stat])

# matrix of frame times in ms for each fluorescent value in the flu matrix
frames_ms = np.empty(flu.shape)
frames_ms.fill(np.nan)

# mark each frame with a time in ms based on its plane
for plane in range(num_planes):
    frame_times = ms_vector[plane::num_planes]
    plane_idx = np.where(cell_plane==plane)[0]
    frames_ms[plane_idx, 0:len(frame_times)] = frame_times

array([             nan,              nan,              nan, ...,
       4056904.32447357, 4057103.58656472,              nan])

In [46]:
lick_start_frames = utils.stim_start_frame(frame_clock=frames_ms,stim_times=run.pre_licks)
lick_idx = [i for i, f in enumerate(frames_ms) if f in lick_start_frames]

lick_frame_bool = np.repeat(False, len(frames_ms))
lick_frame_bool[lick_idx] = True





In [ ]:
preframes_idx = np.argwhere(~np.isnan(frames_ms))
preflu = np.squeeze(flu[:,preframes_idx])
preflu = preflu[:, 1::3] # for plane 1

preframes_ms = np.squeeze(frames_ms[preframes_idx])
preframes_ms = preframes_ms[1::3] # for plane 1

licks = run.pre_licks


In [ ]:
bin_size_ms = 1000
plt.figure()
lick_hist = plt.hist(licks, bins=range(0,max(licks)+bin_size_ms,bin_size_ms))

In [ ]:
onset_bin_ms = utils.threshold_detect(lick_hist[0], 0.5) * bin_size_ms
onset_bin_ms

In [ ]:
first_lick_bin = [min(licks, key=lambda x:abs(x-ms)) for ms in onset_bin_ms]
frame_lick_idx = [utils.closest_frame_before(preframes_ms, lick) for lick in first_lick_bin]

In [ ]:
%%capture

mean_flu = np.mean(preflu, 0)

n_frames_plot = 5

x_axis = np.linspace(-n_frames_plot, n_frames_plot, 2*n_frames_plot)

plt.figure(figsize=(16,16))           
             
for i, idx in enumerate(frame_lick_idx):
    
    plt.subplot(5, int(np.ceil(len(frame_lick_idx)/5)), i+1)
    
    if idx <= n_frames_plot or idx + n_frames_plot > len(mean_flu):
        continue
        
    pre = mean_flu[idx-n_frames_plot:idx]
    post = mean_flu[idx:idx+n_frames_plot]
    
    utils.raster_plot((licks - preframes_ms[idx]) / 1000, y_pos=0, marker='.', color='red')
    
    plt.plot(x_axis, np.hstack((pre, post)))
    plt.ylim((-0.1,0.7))
    plt.xlim((min(x_axis), max(x_axis)))
    

In [ ]:
# meaned bulk trial fluoresence
lick_flu = []
spont_flu = []

trial_len = 5

mean_flu_all = np.mean(flu,0)

final_pre_frame = max(np.ravel(preframes_idx))

for idx in frame_lick_idx:
    
    if idx+trial_len >= len(mean_flu): continue
    
    [lick_flu.append(f) for f in mean_flu[idx:idx+trial_len]]
    
    tstart_spont = idx+final_pre_frame
    [spont_flu.append(f) for f in mean_flu_all[tstart_spont:tstart_spont+trial_len]]

print(np.mean(lick_flu))
print(np.mean(spont_flu))


np.save('/home/jamesrowland/Documents/Code/Vape/lick_flu.npy', lick_flu)
np.save('/home/jamesrowland/Documents/Code/Vape/spont_flu.npy', spont_flu)



In [ ]:
b = plt.boxplot([lick_flu, spont_flu], positions=[1,2], 
                labels=['Licking Trial', 'Spontaneous Trial'],
                showfliers=False)

l = plt.ylabel(r'$\Delta $F/F')
plt.savefig(os.path.join(figure_path, 'lick_boxes.png'), bbox_inches='tight')

In [ ]:
# all flu vales

lick_flu = []
spont_flu = []

for idx in frame_lick_idx:
    
    if idx+trial_len >= len(mean_flu): continue
        
    [lick_flu.append(f) for f in mean_flu[idx:idx+trial_len]]
        
    
    
    print(idx)    


In [ ]:
np.mean(lick_flu)

In [ ]:
idx = np.ravel(frame_lick_idx)[1]


idx+ max(np.ravel(preframes_idx))

In [ ]:
max(preframes_idx)

In [ ]:
idx+trial_len+max(preframes_idx)

In [ ]:
def test_responsive(flu, frame_clock, stim_times, pre_frames = 10, post_frames = 10, offset=0):

    n_frames = flu.shape[1]

    pre_idx = np.repeat(False, n_frames)
    post_idx = np.repeat(False, n_frames)
    # keep track of the previous stim frame to warn against overlap
    prev_frame = 0
                      
    for i, stim_time in enumerate(stim_times):

        stim_frame = utils.closest_frame_before(frame_clock, stim_time) 

        if stim_frame-pre_frames <= 0 or stim_frame+post_frames+offset >= n_frames:
            continue
        elif stim_frame - pre_frames <= prev_frame:
            print('WARNING: STA for stim number {} overlaps with the '
                  'previous stim pre and post arrays can not be '
                  'reshaped to trial by trial'.format(i))
                   
        prev_frame = stim_frame
              
        pre_idx[stim_frame-pre_frames : stim_frame] = True
        post_idx[stim_frame+offset : stim_frame+post_frames+offset] = True
        
    pre = flu[:, pre_idx]
    post = flu[:, post_idx]

    _, pvals = stats.ttest_ind(pre, post, axis=1, paired=True)

    return pre, post, pvals

In [ ]:
pre_frames = 5
post_frames = 5

pre, post, pvals = test_responsive(preflu, preframes_ms, first_lick_bin, pre_frames, post_frames, offset=5)

print(np.mean(post.ravel()) - np.mean(pre.ravel()))


pre = np.reshape(pre, (pre.shape[0], pre_frames, int(pre.shape[1]/pre_frames)))
post = np.reshape(post, (post.shape[0], post_frames, int(post.shape[1]/post_frames)))

assert pre.shape == post.shape
n_trials = pre.shape[2]
mean_pre = np.mean(pre, 2)
mean_post = np.mean(post, 2)





In [ ]:
cell_idx = random.choice(np.where(pvals < 0.05)[0])
cell_idx = 162
print(cell_idx)
save_path = '/home/jamesrowland/Documents/Code/Vape/figures/lick_stas'
# add one so t=0 is at the final point of the pre period 
x_axis = np.arange(-pre_frames, post_frames) / fs + 1/fs
for trial in range(n_trials):
    
    plt.plot(x_axis,
             np.hstack((pre[cell_idx, :, trial], post[cell_idx, :, trial])),
             color='grey', alpha=0.15) 
    
    plt.axvline(x=0, linewidth=5, marker='d',linestyle=':', color='pink')
    plt.text(0.05,1,'Lick Bout Onset',rotation=10, color='pink')
    plt.plot(x_axis, np.hstack((mean_pre[cell_idx, :], mean_post[cell_idx, :])), linewidth=2.5)
    plt.ylim(-1,1)
    plt.xlabel('Time (Seconds)')
    plt.ylabel(r'$\Delta $F/F')
    plt.savefig(os.path.join(save_path, 'lickSTA_cell{}.png'.format(cell_idx)), bbox_inches="tight")

In [ ]:
# correlations






In [ ]:
try:
    a
except ZeroDivisionError as e:
    print(e)


In [ ]:
1/0